In [1]:
import torch
import torchvision
import numpy as np
import cv2 as cv
from torch.utils.mobile_optimizer import optimize_for_mobile

model = torchvision.models.mobilenet_v2(pretrained=True)

model.eval()

MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

In [2]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [3]:
m_mean=[0.485, 0.456, 0.406]
m_std=[0.229, 0.224, 0.225]
opencv_input_image = cv.imread(filename)
res = cv.resize(opencv_input_image,(224, 224), interpolation = cv.INTER_CUBIC)
im_rgb = cv.cvtColor(res, cv.COLOR_BGR2RGB)
im_rgb = np.float32(im_rgb)/255
r,g,b = cv.split(im_rgb)
r = (r - m_mean[0])/m_std[0]
g = (g - m_mean[1])/m_std[1]
b = (b - m_mean[2])/m_std[2]
pre_process = cv.merge((r,g,b))
final_pre_process = cv.dnn.blobFromImage(pre_process)

In [4]:
# sample execution (requires torchvision)
#from PIL import Image
#from torchvision import transforms

#input_image = Image.open(filename)

In [5]:
#preprocess = transforms.Compose([
#    transforms.Resize(224),
#    transforms.ToTensor(),
#    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#])
#input_tensor = preprocess(input_image)
input_tensor = torch.from_numpy(final_pre_process)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
input_batch

tensor([[[[[-1.8610, -1.7754, -1.7583,  ..., -2.1008, -2.1008, -1.8439],
           [-2.0323, -1.8953, -1.9638,  ..., -2.0837, -1.9638, -1.9295],
           [-1.8782, -1.9809, -2.0665,  ..., -2.0494, -2.0323, -1.9809],
           ...,
           [-1.8097, -1.5357, -1.5528,  ..., -1.5357, -1.3130, -1.3302],
           [-1.8268, -1.1418, -1.6213,  ..., -1.6727, -1.3644, -1.1760],
           [-1.1589, -1.0390, -1.6384,  ..., -1.3473, -1.7412, -1.1932]],

          [[-1.6331, -1.6155, -1.6681,  ..., -1.9657, -1.8782, -1.5280],
           [-1.8606, -1.5980, -1.5280,  ..., -2.0357, -2.0007, -1.8256],
           [-1.7731, -1.8431, -1.8957,  ..., -1.9307, -1.9482, -1.8606],
           ...,
           [-0.8978, -0.9503, -0.8277,  ..., -0.9503, -0.5826, -0.4601],
           [-1.2304, -0.6702, -0.8102,  ..., -0.8277, -0.6176, -0.4076],
           [-0.4076, -0.7052, -0.9678,  ..., -0.7402, -1.0553, -0.5301]],

          [[-1.4733, -1.4384, -1.4210,  ..., -1.8044, -1.7870, -1.6302],
           [-1.

In [6]:
#file = open("raw_tensor_data", "wb")
#file.write(input_batch.data_ptr().to_bytes(150528, byteorder='little'))
#file.close()

with torch.no_grad():
    output = model(input_tensor)

torch.nn.functional.softmax(output[0], dim=0)

tensor([3.4588e-07, 1.6758e-08, 2.7807e-08, 1.8452e-08, 4.7866e-08, 5.0434e-07,
        3.3854e-07, 6.0622e-05, 1.1578e-03, 2.9473e-07, 2.6800e-10, 3.6728e-08,
        4.8528e-10, 1.1868e-08, 2.4164e-09, 7.1094e-09, 6.4921e-08, 1.7225e-06,
        5.8309e-07, 1.0582e-08, 1.8381e-08, 6.1718e-09, 3.2813e-08, 1.0519e-07,
        2.2854e-08, 3.8548e-08, 2.5787e-08, 2.9939e-07, 4.3841e-08, 3.8215e-07,
        3.9192e-07, 2.0235e-07, 1.0700e-07, 2.9171e-09, 4.6196e-08, 5.4040e-09,
        5.4424e-08, 1.5754e-08, 6.8187e-08, 2.0042e-06, 3.1404e-08, 9.0537e-09,
        7.4664e-08, 1.3874e-08, 3.6577e-07, 2.1438e-07, 9.0882e-07, 9.4028e-09,
        1.3414e-07, 3.6072e-07, 2.6114e-06, 8.7284e-08, 8.9276e-08, 6.8340e-08,
        1.7829e-07, 1.5330e-08, 1.2911e-07, 1.2032e-08, 1.0370e-07, 2.1345e-08,
        1.2658e-06, 4.7681e-09, 4.8863e-08, 5.0200e-10, 3.3416e-09, 1.1046e-08,
        2.2036e-07, 3.8540e-08, 7.1286e-08, 2.0001e-08, 9.7509e-09, 1.7718e-07,
        8.6142e-09, 7.5555e-09, 2.3858e-

In [7]:
values, indices = torch.max(output[0], 0)
indices

tensor(258)

In [8]:
chan_last_model = model.to(memory_format=torch.channels_last)

In [9]:
input_tensor = torch.from_numpy(final_pre_process)
last_input_tensor = input_tensor.to(memory_format=torch.channels_last)
input_batch = last_input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
with torch.no_grad():
    output = chan_last_model(input_tensor)

torch.nn.functional.softmax(output[0], dim=0)

tensor([3.4588e-07, 1.6758e-08, 2.7807e-08, 1.8452e-08, 4.7866e-08, 5.0434e-07,
        3.3854e-07, 6.0622e-05, 1.1578e-03, 2.9473e-07, 2.6800e-10, 3.6728e-08,
        4.8528e-10, 1.1868e-08, 2.4164e-09, 7.1094e-09, 6.4921e-08, 1.7225e-06,
        5.8309e-07, 1.0582e-08, 1.8381e-08, 6.1718e-09, 3.2813e-08, 1.0519e-07,
        2.2854e-08, 3.8548e-08, 2.5787e-08, 2.9939e-07, 4.3841e-08, 3.8215e-07,
        3.9192e-07, 2.0235e-07, 1.0700e-07, 2.9171e-09, 4.6196e-08, 5.4040e-09,
        5.4424e-08, 1.5754e-08, 6.8187e-08, 2.0042e-06, 3.1404e-08, 9.0537e-09,
        7.4664e-08, 1.3874e-08, 3.6577e-07, 2.1438e-07, 9.0882e-07, 9.4028e-09,
        1.3414e-07, 3.6072e-07, 2.6114e-06, 8.7284e-08, 8.9276e-08, 6.8340e-08,
        1.7829e-07, 1.5330e-08, 1.2911e-07, 1.2032e-08, 1.0370e-07, 2.1345e-08,
        1.2658e-06, 4.7681e-09, 4.8863e-08, 5.0200e-10, 3.3416e-09, 1.1046e-08,
        2.2036e-07, 3.8540e-08, 7.1286e-08, 2.0001e-08, 9.7509e-09, 1.7718e-07,
        8.6142e-09, 7.5555e-09, 2.3858e-

In [10]:
values, indices = torch.max(output[0], 0)
values
indices

tensor(258)

In [22]:
#NCHW Model
scriptedm = torch.jit.script(model)
nchw_script_module_optimized = optimize_for_mobile(scriptedm)
torch.jit.save(nchw_script_module_optimized, "mobilenet_v2.pt")

#NHWC Model
traced_script_module = torch.jit.script(chan_last_model)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
torch.jit.save(traced_script_module_optimized, "mobilenet_v2_nhwc.pt")

#Vulkan NCHW Backend
nchw_script_module_optimized = optimize_for_mobile(scriptedm, backend='vulkan')
torch.jit.save(nchw_script_module_optimized, "mobilenet_v2_vulkan_nchw.pt")

#Vulkan NHWC Backend
traced_script_module_vulkan_optimized = optimize_for_mobile(traced_script_module, backend='vulkan')
torch.jit.save(traced_script_module_vulkan_optimized, "mobilenet_v2_vulkan_nhwc.pt")

In [20]:
x = torch.randn(1, 3, 224, 224, requires_grad=True)

In [13]:
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "mobilenet_v2.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                'output' : {0 : 'batch_size'}})

In [15]:
test_channels_last_model = torch.jit.load("mobilenet_v2_nhwc.pt")

In [27]:
print(test_channels_last_model)

RecursiveScriptModule(original_name=MobileNetV2)


In [28]:
test_channels_last_model.eval()

RecursiveScriptModule(original_name=MobileNetV2)

In [29]:
input_tensor = torch.from_numpy(final_pre_process)
last_input_tensor = input_tensor.to(memory_format=torch.channels_last)
input_batch = last_input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
with torch.no_grad():
    output = test_channels_last_model(input_tensor)

torch.nn.functional.softmax(output[0], dim=0)

/usr/local/lib/python3.9/site-packages/torch/nn/modules/module.py:889: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:934.)
  result = self.forward(*input, **kwargs)


tensor([3.4588e-07, 1.6758e-08, 2.7807e-08, 1.8452e-08, 4.7865e-08, 5.0434e-07,
        3.3854e-07, 6.0621e-05, 1.1578e-03, 2.9473e-07, 2.6800e-10, 3.6727e-08,
        4.8527e-10, 1.1868e-08, 2.4163e-09, 7.1093e-09, 6.4921e-08, 1.7225e-06,
        5.8308e-07, 1.0582e-08, 1.8381e-08, 6.1717e-09, 3.2813e-08, 1.0519e-07,
        2.2853e-08, 3.8548e-08, 2.5787e-08, 2.9939e-07, 4.3840e-08, 3.8215e-07,
        3.9192e-07, 2.0235e-07, 1.0700e-07, 2.9171e-09, 4.6196e-08, 5.4039e-09,
        5.4423e-08, 1.5754e-08, 6.8186e-08, 2.0042e-06, 3.1403e-08, 9.0536e-09,
        7.4663e-08, 1.3874e-08, 3.6577e-07, 2.1438e-07, 9.0880e-07, 9.4027e-09,
        1.3414e-07, 3.6072e-07, 2.6113e-06, 8.7284e-08, 8.9275e-08, 6.8339e-08,
        1.7828e-07, 1.5329e-08, 1.2911e-07, 1.2032e-08, 1.0370e-07, 2.1345e-08,
        1.2658e-06, 4.7681e-09, 4.8862e-08, 5.0200e-10, 3.3416e-09, 1.1046e-08,
        2.2035e-07, 3.8540e-08, 7.1285e-08, 2.0001e-08, 9.7508e-09, 1.7718e-07,
        8.6141e-09, 7.5554e-09, 2.3858e-

In [30]:
values, indices = torch.max(output[0], 0)
values
indices

tensor(258)